In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain_community

In [7]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph


In [28]:
load_dotenv('/content/.env.studiomac', override = True)

True

In [29]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [ ]:
!pip install neo4j

In [30]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [42]:
cypher1 = """
  MATCH (n)
  RETURN count(n)
  """

In [48]:
result1 = kg.query(cypher1)

In [49]:
result1

[{'count(n)': 171}]

In [50]:
print(f"There are {result1[0]['count(n)']} total nodes in the database namely neo4j")

There are 171 total nodes in the database namely neo4j


In [51]:
cypher2 = """
MATCH (n:Movie)
RETURN count(n) AS no_of_movies
"""

In [52]:
result2 = kg.query(cypher2)

In [53]:
result2


[{'no_of_movies': 38}]

In [54]:
print(f"There are {result2[0]['no_of_movies']} movies in the database namely neo4j")

There are 38 movies in the database namely neo4j


In [55]:
cypher3 = """
MATCH (n:Person)
RETURN count(n) AS no_of_people
"""

In [56]:
result3 = kg.query(cypher3)

In [57]:
result3


[{'no_of_people': 133}]

In [58]:
print(f"There are {result3[0]['no_of_people']} people in the database namely neo4j")

There are 133 people in the database namely neo4j


In [59]:
cypher4 = """
MATCH (tom:Person {name:'Tom Hanks'})
RETURN tom
"""

kg.query(cypher4)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

In [63]:
cypher5 = """
MATCH (cloudAtlas:Movie {title:'Cloud Atlas'})
RETURN cloudAtlas
"""

kg.query(cypher5)

[{'cloudAtlas': {'tagline': 'Everything is connected',
   'title': 'Cloud Atlas',
   'released': 2012}}]

In [64]:
cypher6 = """
MATCH (cloudAtlas:Movie {title: 'Cloud Atlas'})
RETURN cloudAtlas.released
"""

kg.query(cypher6)

[{'cloudAtlas.released': 2012}]

In [65]:
cypher7 = """
MATCH (cloudAtlas:Movie {title: 'Cloud Atlas'})
RETURN cloudAtlas.released, cloudAtlas.tagline

"""

kg.query(cypher7)

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

Till now only the exact pattern mathcing is done


#**One can also do the range type queries**

In [73]:
cypher8 = """
Match (genz_movies:Movie)
WHERE genz_movies.released >= 2004
  AND genz_movies.released < 2010
RETURN genz_movies.title

"""

# AND genz_movies.released < 2001
kg.query(cypher8)

[{'genz_movies.title': 'RescueDawn'},
 {'genz_movies.title': 'The Da Vinci Code'},
 {'genz_movies.title': 'V for Vendetta'},
 {'genz_movies.title': 'Speed Racer'},
 {'genz_movies.title': 'Ninja Assassin'},
 {'genz_movies.title': 'Frost/Nixon'},
 {'genz_movies.title': "Charlie Wilson's War"},
 {'genz_movies.title': 'The Polar Express'}]

Pattern matching with multiple nodes

In [77]:
cypher9 = """
MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
RETURN actor.name, movie.title, movie.released
LIMIT 5
"""

kg.query(cypher9)

[{'actor.name': 'Keanu Reeves',
  'movie.title': 'The Matrix',
  'movie.released': 1999},
 {'actor.name': 'Carrie-Anne Moss',
  'movie.title': 'The Matrix',
  'movie.released': 1999},
 {'actor.name': 'Laurence Fishburne',
  'movie.title': 'The Matrix',
  'movie.released': 1999},
 {'actor.name': 'Hugo Weaving',
  'movie.title': 'The Matrix',
  'movie.released': 1999},
 {'actor.name': 'Emil Eifrem',
  'movie.title': 'The Matrix',
  'movie.released': 1999}]

In [78]:
cypher10 = """
MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie {released: 2000})
RETURN actor.name, movie.title, movie.released
LIMIT 5
"""

kg.query(cypher10)

[{'actor.name': 'Tom Cruise',
  'movie.title': 'Jerry Maguire',
  'movie.released': 2000},
 {'actor.name': 'Cuba Gooding Jr.',
  'movie.title': 'Jerry Maguire',
  'movie.released': 2000},
 {'actor.name': 'Renee Zellweger',
  'movie.title': 'Jerry Maguire',
  'movie.released': 2000},
 {'actor.name': 'Kelly Preston',
  'movie.title': 'Jerry Maguire',
  'movie.released': 2000},
 {'actor.name': "Jerry O'Connell",
  'movie.title': 'Jerry Maguire',
  'movie.released': 2000}]

In [82]:
cypher10="""
MATCH (actor:Person {name:"Keanu Reeves"})-[:ACTED_IN]->(reeves_movies:Movie)
RETURN reeves_movies.title,  reeves_movies.released
"""
# LIMIT 3
kg.query(cypher10)

[{'reeves_movies.title': 'The Matrix', 'reeves_movies.released': 1999},
 {'reeves_movies.title': 'The Matrix Reloaded',
  'reeves_movies.released': 2003},
 {'reeves_movies.title': 'The Matrix Revolutions',
  'reeves_movies.released': 2003},
 {'reeves_movies.title': "The Devil's Advocate",
  'reeves_movies.released': 1997},
 {'reeves_movies.title': 'The Replacements', 'reeves_movies.released': 2000},
 {'reeves_movies.title': 'Johnny Mnemonic', 'reeves_movies.released': 1995},
 {'reeves_movies.title': "Something's Gotta Give",
  'reeves_movies.released': 2003}]

In [88]:
cypher11="""
MATCH (actor:Person {name:"Keanu Reeves"})-[:ACTED_IN]->(r)<-[:ACTED_IN]-(coactors)
RETURN coactors.name, r.title
"""
# LIMIT 3
kg.query(cypher11)

[{'coactors.name': 'Carrie-Anne Moss', 'r.title': 'The Matrix'},
 {'coactors.name': 'Laurence Fishburne', 'r.title': 'The Matrix'},
 {'coactors.name': 'Hugo Weaving', 'r.title': 'The Matrix'},
 {'coactors.name': 'Emil Eifrem', 'r.title': 'The Matrix'},
 {'coactors.name': 'Carrie-Anne Moss', 'r.title': 'The Matrix Reloaded'},
 {'coactors.name': 'Laurence Fishburne', 'r.title': 'The Matrix Reloaded'},
 {'coactors.name': 'Hugo Weaving', 'r.title': 'The Matrix Reloaded'},
 {'coactors.name': 'Carrie-Anne Moss', 'r.title': 'The Matrix Revolutions'},
 {'coactors.name': 'Laurence Fishburne', 'r.title': 'The Matrix Revolutions'},
 {'coactors.name': 'Hugo Weaving', 'r.title': 'The Matrix Revolutions'},
 {'coactors.name': 'Charlize Theron', 'r.title': "The Devil's Advocate"},
 {'coactors.name': 'Al Pacino', 'r.title': "The Devil's Advocate"},
 {'coactors.name': 'Brooke Langton', 'r.title': 'The Replacements'},
 {'coactors.name': 'Gene Hackman', 'r.title': 'The Replacements'},
 {'coactors.name': '

# WE CAN CREATE NEW NODES AND RELATIONSHIPS AND ALSO DELETE NODE AND RELATIONSHIPS

In [89]:
cypher12="""
MATCH (actor:Person {name:'Emil Eifrem'})-[acted_in:ACTED_IN]->(movie:Movie)
DELETE acted_in
"""
kg.query(cypher12)

[]

In [92]:
cypher13="""
MATCH (actor:Person {name:'Emil Eifrem'})-[acted_in:ACTED_IN]->(movie:Movie)
RETURN movie
"""

kg.query(cypher13)
# this is for checking whether the relationship still exists or not

[]

In [93]:
cypher14="""
MATCH (actor:Person {name:'Tom Hanks'})-[acted_in:ACTED_IN]->(movie:Movie)
RETURN movie
"""
kg.query(cypher14)

[{'movie': {'tagline': 'At odds in life... in love on-line.',
   'title': "You've Got Mail",
   'released': 1998}},
 {'movie': {'tagline': 'What if someone you never met, someone you never saw, someone you never knew was the only someone for you?',
   'title': 'Sleepless in Seattle',
   'released': 1993}},
 {'movie': {'tagline': 'A story of love, lava and burning desire.',
   'title': 'Joe Versus the Volcano',
   'released': 1990}},
 {'movie': {'tagline': 'In every life there comes a time when that thing you dream becomes that thing you do',
   'title': 'That Thing You Do',
   'released': 1996}},
 {'movie': {'tagline': 'Everything is connected',
   'title': 'Cloud Atlas',
   'released': 2012}},
 {'movie': {'tagline': 'Break The Codes',
   'title': 'The Da Vinci Code',
   'released': 2006}},
 {'movie': {'tagline': "Walk a mile you'll never forget.",
   'title': 'The Green Mile',
   'released': 1999}},
 {'movie': {'tagline': 'Houston, we have a problem.',
   'title': 'Apollo 13',
   'rel

In [106]:
cypher15="""
CREATE (actor:Person {name:'Dhruv'})
RETURN actor
"""
kg.query(cypher15)

[{'actor': {'name': 'Dhruv'}}]

In [107]:
cypher16="""
MATCH (actor:Person {name:'Dhruv'}),(m:Movie {title:"Cast Away"})
MERGE (actor)-[act:ACTED_IN]->(m)
RETURN actor,act,m
"""
kg.query(cypher16)

[{'actor': {'name': 'Dhruv'},
  'act': ({'name': 'Dhruv'},
   'ACTED_IN',
   {'tagline': 'At the edge of the world, his journey begins.',
    'title': 'Cast Away',
    'released': 2000}),
  'm': {'tagline': 'At the edge of the world, his journey begins.',
   'title': 'Cast Away',
   'released': 2000}}]

In [108]:
cypher17="""
MATCH (actor:Person)-[acted_in:ACTED_IN]->(movie:Movie {title:'Cast Away'})
RETURN actor
"""
kg.query(cypher17)

[{'actor': {'born': 1963, 'name': 'Helen Hunt'}},
 {'actor': {'born': 1956, 'name': 'Tom Hanks'}},
 {'actor': {'name': 'Dhruv'}}]

In [103]:
cypher12="""
MATCH (actor:Person {name:'Dhruv'})-[acted_in:ACTED_IN]->(movie:Movie)
DELETE actor,acted_in
"""
kg.query(cypher12)

# here i am deleting both the relationship and the node

[]